In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import (
    mean_squared_error,
    roc_auc_score,
    r2_score,
)

DATA_DIR = Path("data")  # change to your local path
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# ---- 1.1 Load wildfires ----
fires_path = DATA_DIR / "wildfires_fires.parquet"  # adjust
fires = pd.read_parquet(
    fires_path,
    columns=["FIRE_SIZE", "DISCOVERY_DATE", "STATE", "STAT_CAUSE_CODE"],
)

# Convert DISCOVERY_DATE to datetime.
# If it's already datetime, this will do nothing.
# The error indicates that DISCOVERY_DATE is already datetime.date objects,
# but not np.datetime64, so `unit='D', origin='julian'` is incompatible.
# We can simply convert them to np.datetime64 directly.
if not np.issubdtype(fires["DISCOVERY_DATE"].dtype, np.datetime64):
    fires["DISCOVERY_DATE"] = pd.to_datetime(fires["DISCOVERY_DATE"])

fires = fires.rename(columns={"DISCOVERY_DATE": "date"})

# Filter date range 1992–2015 (dataset range)
fires = fires[(fires["date"].dt.year >= 1992) & (fires["date"].dt.year <= 2015)]

fires.head()

In [ ]:
# ---- 1.2 Aggregate to state–day ----

# Basic daily targets
daily_agg = (
    fires.groupby(["STATE", "date"])
    .agg(
        n_fires=("FIRE_SIZE", "size"),
        area_burned=("FIRE_SIZE", "sum"),
    )
    .reset_index()
)

# Cause counts per state–day (wide format)
cause_counts = (
    fires.pivot_table(
        index=["STATE", "date"],
        columns="STAT_CAUSE_CODE",
        values="FIRE_SIZE",  # any column, we just count
        aggfunc="count",
        fill_value=0,
    )
    .rename_axis(columns="cause_code")
)

# Make nicer column names: cause_1, cause_2, ...
cause_counts.columns = [f"cause_{int(c)}" for c in cause_counts.columns]

cause_counts = cause_counts.reset_index()

# Merge counts into daily_agg
fires_daily = daily_agg.merge(
    cause_counts, on=["STATE", "date"], how="left"
).fillna(0)

fires_daily["any_fire"] = (fires_daily["n_fires"] > 0).astype(int)

fires_daily.head()

In [ ]:
# ---- 2.1 Load weather ----
weather_path = DATA_DIR / "us_daily_weather_1992_2015.parquet"  # adjust
weather = pd.read_parquet(weather_path)

# Load cities data to get state codes
cities_path = DATA_DIR / "cities.csv"
cities_df = pd.read_csv(cities_path)

# Adapt these column names to your file
# Here we assume:
#   'date' (datetime), 'state' for 2-letter US state (corrected from state_code)
#   'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wspd', 'pres'
weather["date"] = pd.to_datetime(weather["date"])

# Restrict to 1992–2015 and US
weather = weather[(weather["date"].dt.year >= 1992) & (weather["date"].dt.year <= 2015)]
# If you already filtered to US, skip this; otherwise:
# weather = weather[weather["country_code"] == "US"]

# Merge weather with cities to get state information
weather = weather.merge(
    cities_df[["city_name", "state"]].drop_duplicates(), on="city_name", how="left"
)

# Rename the 'state' column from cities_df to 'STATE' to match fires_daily
weather = weather.rename(columns={"state": "STATE",
                                  "avg_temp_c" : "tavg",
                                  "min_temp_c" : "tmin",
                                  "max_temp_c" : "tmax",
                                  "precipitation_mm" : "prcp",
                                  "snow_depth_mm" : "snow",
                                  "avg_wind_speed_kmh" : "wspd",
                                  "avg_sea_level_pres_hpa" : "pres",})


weather.head()

In [ ]:
# ---- 2.2 Aggregate station data to state–day ----

weather_daily = (
    weather.groupby(["STATE", "date"])
    .agg(
        tavg=("tavg", "mean"),
        tmin=("tmin", "mean"),
        tmax=("tmax", "mean"),
        prcp=("prcp", "sum"),
        snow=("snow", "sum"),
        wspd=("wspd", "mean"),
        pres=("pres", "mean"),
    )
    .reset_index()
)

weather_daily.head()

In [ ]:
# ---- 2.3 Add calendar features and lagged weather features ----

def add_lagged_features(
    df,
    group_col,
    date_col,
    base_cols,
    windows=(3, 7, 30),
    shift=1,
):
    """
    For each base_col, compute rolling means or sums (depending on column name)
    over given windows, then shift by `shift` days to avoid using same-day info.
    """
    df = df.sort_values([group_col, date_col]).copy()

    for col in base_cols:
        for w in windows:
            roll = (
                df.groupby(group_col)[col]
                .transform(lambda x: x.rolling(w, min_periods=1).mean())
            )
            df[f"{col}_mean_{w}"] = roll.shift(shift)

    # Example of cumulative precipitation (sums instead of means)
    for w in [7, 30]:
        roll = (
            df.groupby(group_col)["prcp"]
            .transform(lambda x: x.rolling(w, min_periods=1).sum())
        )
        df[f"prcp_sum_{w}"] = roll.shift(shift)

    return df


weather_feat = weather_daily.copy()

# State name to abbreviation mapping to align with fires_daily
state_name_to_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC'
}


# Apply the mapping to weather_feat['STATE']
weather_feat['STATE'] = weather_feat['STATE'].map(state_name_to_abbrev)

# Drop rows where state mapping resulted in NaN (if any states were not in our map)
weather_feat.dropna(subset=['STATE'], inplace=True)

weather_feat["year"] = weather_feat["date"].dt.year
weather_feat["month"] = weather_feat["date"].dt.month
weather_feat["doy"] = weather_feat["date"].dt.dayofyear

base_cols = ["tavg", "tmin", "tmax", "prcp", "wspd", "pres"]
weather_feat = add_lagged_features(
    weather_feat,
    group_col="STATE",
    date_col="date",
    base_cols=base_cols,
)

# Drop first few rows where lags are NaN
weather_feat = weather_feat.dropna().reset_index(drop=True)

weather_feat.head()

In [ ]:
# ---- 2.3 Add calendar features and lagged weather features ----

def add_lagged_features(
    df,
    group_col,
    date_col,
    base_cols,
    windows=(3, 7, 30),
    shift=1,
):
    """
    For each base_col, compute rolling means or sums (depending on column name)
    over given windows, then shift by `shift` days to avoid using same-day info.
    """
    df = df.sort_values([group_col, date_col]).copy()

    for col in base_cols:
        for w in windows:
            roll = (
                df.groupby(group_col)[col]
                .transform(lambda x: x.rolling(w, min_periods=1).mean())
            )
            df[f"{col}_mean_{w}"] = roll.shift(shift)

    # Example of cumulative precipitation (sums instead of means)
    for w in [7, 30]:
        roll = (
            df.groupby(group_col)["prcp"]
            .transform(lambda x: x.rolling(w, min_periods=1).sum())
        )
        df[f"prcp_sum_{w}"] = roll.shift(shift)

    return df


weather_feat = weather_daily.copy()

# State name to abbreviation mapping to align with fires_daily
state_name_to_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

# Apply the mapping to weather_feat['STATE']
weather_feat['STATE'] = weather_feat['STATE'].map(state_name_to_abbrev)

# Drop rows where state mapping resulted in NaN (if any states were not in our map)
weather_feat.dropna(subset=['STATE'], inplace=True)

# Impute any remaining NaNs in weather features before adding lagged features
# This ensures lagged features don't become NaN due to missing base values.
# Explicitly set 'STATE' as index for fill, then reset to column
weather_feat = weather_feat.set_index('STATE')
weather_feat = weather_feat.groupby(level=0).ffill().bfill()
weather_feat = weather_feat.reset_index()

weather_feat["year"] = weather_feat["date"].dt.year
weather_feat["month"] = weather_feat["date"].dt.month
weather_feat["doy"] = weather_feat["date"].dt.dayofyear

base_cols = ["tavg", "tmin", "tmax", "prcp", "wspd", "pres"]
weather_feat = add_lagged_features(
    weather_feat,
    group_col="STATE",
    date_col="date",
    base_cols=base_cols,
)

# Drop first few rows where lags are NaN (expected from the shift operation)
weather_feat = weather_feat.dropna().reset_index(drop=True)

weather_feat.head()

In [ ]:
data = weather_feat.merge(
    fires_daily, on=["STATE", "date"], how="left"
)

# Replace NaNs in fire-related columns with 0
fire_cols = [c for c in data.columns if c.startswith("cause_")] + [
    "n_fires",
    "area_burned",
    "any_fire",
]
for c in fire_cols:
    if c in data.columns:
        data[c] = data[c].fillna(0)

# Add transformed target for area
data["log_area_burned"] = np.log1p(data["area_burned"])

data.head()

In [ ]:
# ---- 4.1 Choose features for clustering ----
cluster_features = [
    "tavg_mean_7",
    "tavg_mean_30",
    "prcp_sum_7",
    "prcp_sum_30",
    "wspd_mean_7",
    "pres_mean_7",
]

# Some columns may not exist depending on your base_cols / windows;
# filter to those that actually exist.
cluster_features = [c for c in cluster_features if c in data.columns]

# ---- 4.2 Fit KMeans on training-years only ----
train_year_cutoff = 2011
train_mask = data["year"] < train_year_cutoff

scaler = StandardScaler()
X_cluster_train = scaler.fit_transform(data.loc[train_mask, cluster_features])

kmeans = KMeans(n_clusters=5, random_state=RANDOM_STATE, n_init=10)
kmeans.fit(X_cluster_train)

# Assign cluster to all rows
data["weather_cluster"] = kmeans.predict(
    scaler.transform(data[cluster_features])
)

data["weather_cluster"].value_counts().sort_index()

In [ ]:
# Label-encode states
le_state = LabelEncoder()
data["STATE_LE"] = le_state.fit_transform(data["STATE"])

# Final feature list (excluding targets)
exclude_cols = {
    "n_fires",
    "any_fire",
    "area_burned",
    "log_area_burned",
    "STATE",
    "date",
} | set([c for c in data.columns if c.startswith("cause_")])

feature_cols = [c for c in data.columns if c not in exclude_cols]
len(feature_cols), feature_cols[:10]

In [ ]:
# Train / test split by year (no leakage from future to past)
train_mask = data["year"] < train_year_cutoff
test_mask = ~train_mask

X_train = data.loc[train_mask, feature_cols]
X_test = data.loc[test_mask, feature_cols]

y_train_count = data.loc[train_mask, "n_fires"]
y_test_count = data.loc[test_mask, "n_fires"]

y_train_bin = data.loc[train_mask, "any_fire"]
y_test_bin = data.loc[test_mask, "any_fire"]

y_train_log_area = data.loc[train_mask, "log_area_burned"]
y_test_log_area = data.loc[test_mask, "log_area_burned"]

In [ ]:
# ---- 6.1 Count model (Poisson) ----

lgb_train = lgb.Dataset(X_train, label=y_train_count)
lgb_valid = lgb.Dataset(X_test, label=y_test_count)

params_count = {
    "objective": "poisson",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "metric": "rmse",
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "seed": RANDOM_STATE,
}

model_count = lgb.train(
    params_count,
    lgb_train,
    num_boost_round=2000,
    valid_sets=[lgb_valid],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)],
)

pred_count = model_count.predict(X_test, num_iteration=model_count.best_iteration)
# Fix: Remove squared=False and take np.sqrt to get RMSE
rmse_count = np.sqrt(mean_squared_error(y_test_count, pred_count))

print(f"Count model RMSE: {rmse_count:.3f}")
print(f"Mean actual count in test: {y_test_count.mean():.3f}")

In [ ]:
# ---- 6.2 Binary any-fire model (for AUC) ----

params_bin = {
    "objective": "binary",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "metric": "auc",
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "seed": RANDOM_STATE,
}

lgb_train_bin = lgb.Dataset(X_train, label=y_train_bin)
lgb_valid_bin = lgb.Dataset(X_test, label=y_test_bin)

model_bin = lgb.train(
    params_bin,
    lgb_train_bin,
    num_boost_round=2000,
    valid_sets=[lgb_valid_bin],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)],
)

prob_any_fire = model_bin.predict(
    X_test, num_iteration=model_bin.best_iteration
)
auc_any_fire = roc_auc_score(y_test_bin, prob_any_fire)

print(f"AUC (any_fire): {auc_any_fire:.3f}")
print(f"Positive rate (fires on a day): {y_test_bin.mean():.3f}")

In [ ]:
# ---- 6.3 Simple clustering-only baseline for counts ----
# Predict the mean count for each weather_cluster on train,
# then apply to test based only on cluster label.

cluster_means = (
    data.loc[train_mask]
    .groupby("weather_cluster")["n_fires"]
    .mean()
)

baseline_pred = data.loc[test_mask, "weather_cluster"].map(cluster_means)
baseline_rmse = np.sqrt(mean_squared_error(
    y_test_count, baseline_pred
))

print(f"Cluster baseline RMSE: {baseline_rmse:.3f}")
print(f"LightGBM Poisson RMSE: {rmse_count:.3f}")

In [ ]:
# ---- 7.1 Area model ----

params_area = {
    "objective": "regression",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "metric": "rmse",
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "seed": RANDOM_STATE,
}

lgb_train_area = lgb.Dataset(X_train, label=y_train_log_area)
lgb_valid_area = lgb.Dataset(X_test, label=y_test_log_area)

model_area = lgb.train(
    params_area,
    lgb_train_area,
    num_boost_round=2000,
    valid_sets=[lgb_valid_area],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)],
)

pred_log_area = model_area.predict(
    X_test, num_iteration=model_area.best_iteration
)

rmse_log_area = np.sqrt(mean_squared_error(
    y_test_log_area, pred_log_area
))
r2_log_area = r2_score(y_test_log_area, pred_log_area)

print(f"Area model RMSE (log1p): {rmse_log_area:.3f}")
print(f"Area model R^2 (log1p): {r2_log_area:.3f}")

# Baseline: predict mean log-area of train
baseline_log = np.full_like(y_test_log_area, y_train_log_area.mean())
baseline_rmse_log = np.sqrt(mean_squared_error(
    y_test_log_area, baseline_log
))
print(f"Baseline RMSE (log1p mean): {baseline_rmse_log:.3f}")

In [ ]:
# ---- 8.1 Prepare cause targets ----

cause_cols = [c for c in data.columns if c.startswith("cause_")]
len(cause_cols), cause_cols

In [ ]:
# Keep only days with at least one fire
data_cause = data[data["n_fires"] > 0].copy()

# Proportions per cause
for c in cause_cols:
    data_cause[c + "_prop"] = data_cause[c] / data_cause["n_fires"]

target_prop_cols = [c + "_prop" for c in cause_cols]

# Train/test split by year (same cutoff)
train_mask_cause = data_cause["year"] < train_year_cutoff
test_mask_cause = ~train_mask_cause

X_train_cause = data_cause.loc[train_mask_cause, feature_cols]
X_test_cause = data_cause.loc[test_mask_cause, feature_cols]

Y_train_cause = data_cause.loc[train_mask_cause, target_prop_cols].values
Y_test_cause = data_cause.loc[test_mask_cause, target_prop_cols].values

In [ ]:
# Load only STAT_CAUSE_CODE and STAT_CAUSE_DESCR to create a mapping
# fires_path is already defined in the notebook.
fires_cause_descriptions = pd.read_parquet(
    fires_path,
    columns=["STAT_CAUSE_CODE", "STAT_CAUSE_DESCR"]
)

# Create a unique mapping from code to description
# Handle potential duplicates by taking the first description for each code
cause_code_to_descr = fires_cause_descriptions[['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']].drop_duplicates().set_index('STAT_CAUSE_CODE')['STAT_CAUSE_DESCR'].to_dict()

print("Cause Code to Description Mapping:")
for code, descr in sorted(cause_code_to_descr.items()):
    print(f"{code}: {descr}")

In [ ]:
# Prepare the renaming dictionary
rename_dict = {}
for col in data.columns:
    if col.startswith("cause_"):
        cause_code_str = col.split('_')[1] # Extract the numeric code as string
        # Check if the extracted string is actually a digit before converting to int
        if cause_code_str.isdigit():
            cause_code = int(cause_code_str)
            if cause_code in cause_code_to_descr:
                # Create a clean, descriptive column name
                # Replace spaces/special chars with underscores and convert to lowercase
                new_col_name = "cause_" + cause_code_to_descr[cause_code].replace(" ", "_").replace("/", "_").replace("-", "_").replace("(","").replace(")","").lower()
                rename_dict[col] = new_col_name
            else:
                print(f"Warning: No description found for STAT_CAUSE_CODE {cause_code}. Column '{col}' will not be renamed.")
        else:
            # If it's not a digit, it's likely already renamed or not a standard numeric cause column.
            # We can skip renaming it.
            print(f"Info: Column '{col}' is not in 'cause_X' numeric format; skipping renaming.")

# Apply the renaming to the DataFrame
data = data.rename(columns=rename_dict)

# Display the head of the DataFrame to show the new column names
print("DataFrame with renamed cause columns:")
display(data.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(y_train_log_area, bins=50, kde=True)
plt.title('Distribution of log_area_burned (Training Set) - Recheck')
plt.xlabel('log(Area Burned + 1)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(y_test_log_area, bins=50, kde=True)
plt.title('Distribution of log_area_burned (Test Set) - Recheck')
plt.xlabel('log(Area Burned + 1)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print('Summary statistics for y_train_log_area (Recheck):')
print(y_train_log_area.describe())

In [ ]:
print('\nSummary statistics for y_test_log_area (Recheck):')
print(y_test_log_area.describe())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(y_train_log_area, bins=50, kde=True)
plt.title('Distribution of log_area_burned (Training Set)')
plt.xlabel('log(Area Burned + 1)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(y_test_log_area, bins=50, kde=True)
plt.title('Distribution of log_area_burned (Test Set)')
plt.xlabel('log(Area Burned + 1)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print('Summary statistics for y_train_log_area:')
print(y_train_log_area.describe())

In [ ]:
print('\nSummary statistics for y_test_log_area:')
print(y_test_log_area.describe())